In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.model_selection import train_test_split

# Constants

In [2]:
alpha = -0.1

# Preprocessing

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)
pd.options.display.max_columns = 100

In [4]:
%%time
df = pd.read_csv('data/train_2011_2012_2013.csv', sep=";", nrows=1e10)

<string>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 40s


In [5]:
df.head()

,DATE,DAY_OFF,DAY_DS,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ACD_COD,ACD_LIB,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_ACWOUTADJCALLS,CSPL_AUXOUTADJCALLS,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_DISCCALLS,CSPL_DISCTIME,CSPL_INTRVL,CSPL_INCOMPLETE,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,855,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1587,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,30,0,0,15,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1589,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1591,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,10800,832,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1555,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,10,10,1,0,7,0,0,0,0,0,0,30,0,1,15,0,0,0,1,0,1,0,488


In [6]:
target = "CSPL_RECEIVED_CALLS"

### deleting all columns but very few

In [7]:
df = df[['DATE', target, 'ASS_ASSIGNMENT']]

In [8]:
df.head()

,DATE,CSPL_RECEIVED_CALLS,ASS_ASSIGNMENT
0,2011-04-24 01:30:00.000,0,Téléphonie
1,2011-04-24 01:30:00.000,0,Téléphonie
2,2011-04-24 01:30:00.000,0,Téléphonie
3,2011-04-24 01:30:00.000,0,Téléphonie
4,2011-04-24 01:30:00.000,1,Téléphonie


In [9]:
df.DATE = pd.to_datetime(df.DATE)

In [10]:
import datetime

get the time slot of the day (0 for 00:00, 1 for 00:30, ... 47 for 23:30)

In [11]:
def get_slot(timestamp):
    assert timestamp.minute in [0, 30]
    return 2 * timestamp.hour + (timestamp.minute == 30)

In [12]:
%%time
df['slot'] = df.DATE.apply(get_slot)

Wall time: 33.5 s


In [13]:
%%time
df['dayofweek'] = df.DATE.apply(lambda x: x.dayofweek)

Wall time: 33.2 s


In [14]:
%%time
df['ass_hashed'] = df.ASS_ASSIGNMENT.apply(hash)

Wall time: 1.82 s


In [15]:
df.head()

,DATE,CSPL_RECEIVED_CALLS,ASS_ASSIGNMENT,slot,dayofweek,ass_hashed
0,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
1,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
2,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
3,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
4,2011-04-24 01:30:00,1,Téléphonie,3,6,-1730673935


In [16]:
X_df = df[['ass_hashed', 'slot', 'dayofweek']]
y_df = df[target]

# Training

In [17]:
from sklearn.model_selection import ShuffleSplit

skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)
skf_is = list(skf.split(X_df))[0]
train_is, test_is = skf_is

X_train_df = X_df.iloc[train_is].copy()                                  
y_train_df = y_df.iloc[train_is].copy()
X_test_df = X_df.iloc[test_is].copy()                                    
y_test_df = y_df.iloc[test_is].copy() 

In [26]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

In [27]:
%%time
rf.fit(X_train_df, y_train_df)

Wall time: 3min 20s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [33]:
y_pred_df = rf.predict(X_test_df)

# Evaluation

In [34]:
def lin_ex_scalar(y, y_hat):
    return np.exp(alpha * (y - y_hat)) - alpha * (y - y_hat) - 1

In [35]:
def lin_ex(y_df, y_est_df):
    return lin_ex_scalar(y_df, y_est_df)

In [36]:
error = lin_ex(y_test_df, y_pred_df)

In [37]:
error.mean

<bound method Series.mean of 4049878     0.002436
9350019     0.002617
5850100     0.000009
1068964     0.001397
9174709     0.000005
1240207     0.000006
3398764     0.000011
3326100     0.000006
4141728     0.004506
10311794    0.003489
5140058     0.000158
4685338     0.000016
9238946     0.000049
7873925     0.004421
331540      0.047637
6660886     0.000557
6529930     2.089180
5163087     0.002439
3407089     0.000003
711504      0.003037
6048378     0.001066
4801023     0.000001
1645413     0.000456
3014729     0.000266
3759237     0.003111
4979110     0.004777
5351777     0.002796
3210684     0.000040
9932560     0.002055
7385950     0.000015
              ...   
2472840     0.005334
8246570     0.005715
1221008     0.000009
1641903     0.001853
3750399     0.003486
929900      0.011146
8578063     0.000616
3243281     0.000026
998076      0.002461
9488346     0.000174
9286985     0.000061
3981840     0.002616
6449031     0.003823
7852761     0.004765
4095292     0.000021
88959

# Read submission data

In [38]:
sub = pd.read_csv('data/submission.txt', sep='\t')

In [39]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28 00:00:00.000,CMS,0
1,2012-12-28 00:00:00.000,Crises,0
2,2012-12-28 00:00:00.000,Domicile,0
3,2012-12-28 00:00:00.000,Gestion,0
4,2012-12-28 00:00:00.000,Gestion - Accueil Telephonique,0


In [40]:
sub.DATE = pd.to_datetime(sub.DATE)

In [41]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28,CMS,0
1,2012-12-28,Crises,0
2,2012-12-28,Domicile,0
3,2012-12-28,Gestion,0
4,2012-12-28,Gestion - Accueil Telephonique,0


In [42]:
# there are no training data for the submission dates!
sub_ix = sub.set_index('DATE').index
df_ix = df.set_index("DATE").index
df_ix.intersection(sub_ix)

DatetimeIndex([], dtype='datetime64[ns]', name=u'DATE', freq=None)

In [43]:
sub.shape

(82909, 3)

In [44]:
sub['dayofweek'] = sub.DATE.apply(lambda x: x.dayofweek)
sub['slot'] = sub.DATE.apply(get_slot)

In [45]:
sub['ass_hashed'] = sub.ASS_ASSIGNMENT.apply(hash)

In [46]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot,ass_hashed
0,2012-12-28,CMS,0,4,0,-1466160996
1,2012-12-28,Crises,0,4,0,-272063095
2,2012-12-28,Domicile,0,4,0,-1965427828
3,2012-12-28,Gestion,0,4,0,822364246
4,2012-12-28,Gestion - Accueil Telephonique,0,4,0,2096569015


In [47]:
X_sub = sub[['ass_hashed', 'slot', 'dayofweek']]

In [48]:
sub['prediction'] = rf.predict(X_sub)

In [49]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot,ass_hashed
0,2012-12-28,CMS,0.00000,4,0,-1466160996
1,2012-12-28,Crises,0.00000,4,0,-272063095
2,2012-12-28,Domicile,0.03664,4,0,-1965427828
3,2012-12-28,Gestion,0.00000,4,0,822364246
4,2012-12-28,Gestion - Accueil Telephonique,0.01048,4,0,2096569015


### cell below takes ~3 hours to run

In [50]:
sub.prediction.describe()

count    82909.000000
mean         1.335820
std          4.700776
min          0.000000
25%          0.000000
50%          0.095266
75%          0.694154
max         62.521648
Name: prediction, dtype: float64

In [51]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot,ass_hashed
0,2012-12-28,CMS,0.00000,4,0,-1466160996
1,2012-12-28,Crises,0.00000,4,0,-272063095
2,2012-12-28,Domicile,0.03664,4,0,-1965427828
3,2012-12-28,Gestion,0.00000,4,0,822364246
4,2012-12-28,Gestion - Accueil Telephonique,0.01048,4,0,2096569015


In [52]:
real_sub = sub.iloc[:, :3]

In [53]:
real_sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28,CMS,0.00000
1,2012-12-28,Crises,0.00000
2,2012-12-28,Domicile,0.03664
3,2012-12-28,Gestion,0.00000
4,2012-12-28,Gestion - Accueil Telephonique,0.01048


In [54]:
real_sub.DATE = real_sub.DATE.astype(str) + ".000"

In [55]:
real_sub.to_csv('data/sub_chiaman_2.txt', sep='\t', index=None)